# Enable Agent Tutorial Part 2: ML 모델의 Enable Agent화

## 개요

Part 1에서 학습한 ML 모델을 Enable Agent로 변환한다.

## Enable Agent란?

Enable Agent는 기존 ML/DL 모델을 LLM 기반 에이전트 시스템에 통합하는 패턴이다.

### 기존 방식 vs Enable Agent

**기존 방식:**
```python
X = np.array([[5.1, 3.5, 1.4, 0.2]])
model = joblib.load('model.pkl')
prediction = model.predict(X)  # [0] - 숫자만 반환
```

**Enable Agent 방식:**
```python
agent = IrisEnableAgent('skill.yaml')
result = agent.predict({
    "sepal_length": 5.1,
    "sepal_width": 3.5,
    "petal_length": 1.4,
    "petal_width": 0.2
})
# 결과: 예측 품종, 신뢰도, 확률 분포, 특성 중요도, AI 설명
```

## 학습 내용

- YAML 스킬 파일 작성
- Agent 클래스 구현
- OpenAI Function Calling 통합
- 결과 설명 생성

---
## 1. 환경 설정

In [5]:
import os
import json
import yaml
import joblib
import numpy as np
from datetime import datetime
from typing import Dict, Any, Tuple
from dotenv import load_dotenv
from openai import OpenAI

In [6]:
# .env 파일에서 환경 변수를 로드한다
!echo "OPENAI_API_KEY=sk-proj-OpenAI API Key 붙혀넣기" >> .env

load_dotenv()

True

---
## 2. YAML 스킬 파일 작성

스킬 파일은 Agent의 능력, 입출력 스키마, 제약조건 등을 선언적으로 정의한다.

### 스킬 파일의 주요 구성요소

- **기본 정보**: Agent 이름, 버전, 설명
- **능력 정의**: 주요 기능과 부가 기능
- **모델 정보**: 모델 타입, 경로
- **입출력 스키마**: JSON Schema 형식으로 정의
- **사용 예제**: 테스트 케이스
- **제약조건**: 성능/보안 제약

In [7]:
skill_content = '''agent_name: "Iris Classification Agent"
version: "1.0.0"
description: "붓꽃(Iris) 품종을 분류하는 머신러닝 Enable Agent"

capabilities:
  primary: "iris_classification"
  secondary:
    - "feature_analysis"
    - "prediction_explanation"

model_info:
  type: "RandomForestClassifier"
  framework: "scikit-learn"
  model_path: "models/iris_classifier.pkl"
  metadata_path: "models/iris_classifier_metadata.json"

input_schema:
  type: "object"
  required:
    - "sepal_length"
    - "sepal_width"
    - "petal_length"
    - "petal_width"
  properties:
    sepal_length:
      type: "number"
      description: "꽃받침 길이 (cm)"
      minimum: 4.0
      maximum: 8.0
    sepal_width:
      type: "number"
      description: "꽃받침 너비 (cm)"
      minimum: 2.0
      maximum: 5.0
    petal_length:
      type: "number"
      description: "꽃잎 길이 (cm)"
      minimum: 1.0
      maximum: 7.0
    petal_width:
      type: "number"
      description: "꽃잎 너비 (cm)"
      minimum: 0.1
      maximum: 3.0

output_schema:
  type: "object"
  properties:
    predicted_class:
      type: "string"
      description: "예측된 붓꽃 품종"
    confidence:
      type: "number"
      description: "예측 신뢰도 (0-1)"
    probabilities:
      type: "object"
      description: "각 클래스별 확률"
    feature_importance:
      type: "object"
      description: "특성별 중요도"

usage_examples:
  - description: "Setosa 품종 예측"
    input:
      sepal_length: 5.1
      sepal_width: 3.5
      petal_length: 1.4
      petal_width: 0.2
    expected_output: "setosa"
  - description: "Versicolor 품종 예측"
    input:
      sepal_length: 6.4
      sepal_width: 3.2
      petal_length: 4.5
      petal_width: 1.5
    expected_output: "versicolor"

constraints:
  max_requests_per_minute: 100
  timeout_seconds: 5
'''

os.makedirs('skills', exist_ok=True)
with open('skills/iris_agent_skill.yaml', 'w', encoding='utf-8') as f:
    f.write(skill_content)

print("스킬 파일 생성 완료: skills/iris_agent_skill.yaml")
print("\n스킬 파일 내용:")
print(skill_content)

스킬 파일 생성 완료: skills/iris_agent_skill.yaml

스킬 파일 내용:
agent_name: "Iris Classification Agent"
version: "1.0.0"
description: "붓꽃(Iris) 품종을 분류하는 머신러닝 Enable Agent"

capabilities:
  primary: "iris_classification"
  secondary:
    - "feature_analysis"
    - "prediction_explanation"

model_info:
  type: "RandomForestClassifier"
  framework: "scikit-learn"
  model_path: "models/iris_classifier.pkl"
  metadata_path: "models/iris_classifier_metadata.json"

input_schema:
  type: "object"
  required:
    - "sepal_length"
    - "sepal_width"
    - "petal_length"
    - "petal_width"
  properties:
    sepal_length:
      type: "number"
      description: "꽃받침 길이 (cm)"
      minimum: 4.0
      maximum: 8.0
    sepal_width:
      type: "number"
      description: "꽃받침 너비 (cm)"
      minimum: 2.0
      maximum: 5.0
    petal_length:
      type: "number"
      description: "꽃잎 길이 (cm)"
      minimum: 1.0
      maximum: 7.0
    petal_width:
      type: "number"
      description: "꽃잎 너비 (cm)"
      minim

---
## 3. Enable Agent 클래스 구현

Enable Agent는 다음 핵심 기능을 제공한다.

### Agent 래핑 레이어

```
+-------------------------------------+
|     IrisEnableAgent (Wrapper)       |
+-------------------------------------+
| 1. 입력 검증 (validate_input)       |
| 2. 모델 실행 (predict)              |
| 3. 결과 해석 (explain_prediction)   |
| 4. Tool 정의 (generate_tool_def)    |
+-------------------------------------+
              |
              v
+-------------------------------------+
|     RandomForestClassifier          |
|     (Original ML Model)             |
+-------------------------------------+
```

### 주요 메서드

- `__init__()`: 스킬 파일과 모델 로드
- `get_capability_description()`: Agent 능력을 자연어로 설명
- `validate_input()`: 입력 데이터 유효성 검증
- `predict()`: 예측 수행 및 풍부한 결과 반환
- `explain_prediction()`: LLM으로 결과 설명 생성
- `generate_tool_definition()`: OpenAI Function Calling용 정의

In [8]:
class IrisEnableAgent:
    """Iris 분류 모델을 Enable Agent로 래핑한 클래스"""
    
    def __init__(self, skill_path: str):
        """Enable Agent 초기화"""
        with open(skill_path, 'r', encoding='utf-8') as f:
            self.skill = yaml.safe_load(f)
        
        model_path = self.skill['model_info']['model_path']
        self.model = joblib.load(model_path)
        
        metadata_path = self.skill['model_info']['metadata_path']
        with open(metadata_path, 'r', encoding='utf-8') as f:
            self.metadata = json.load(f)
        
        self.client = OpenAI()
        
        print(f"Enable Agent 초기화 완료: {self.skill['agent_name']}")
        print(f"모델 정확도: {self.metadata['accuracy']:.4f}")
    
    def get_capability_description(self) -> str:
        """LLM이 이해할 수 있는 형태로 Agent 기능을 설명한다"""
        description = f"""
Agent 이름: {self.skill['agent_name']}
버전: {self.skill['version']}
설명: {self.skill['description']}

주요 기능: {self.skill['capabilities']['primary']}
부가 기능: {', '.join(self.skill['capabilities']['secondary'])}

입력 파라미터:
- sepal_length: 꽃받침 길이 (cm, 4.0-8.0)
- sepal_width: 꽃받침 너비 (cm, 2.0-5.0)
- petal_length: 꽃잎 길이 (cm, 1.0-7.0)
- petal_width: 꽃잎 너비 (cm, 0.1-3.0)

출력:
- predicted_class: 예측된 붓꽃 품종 (setosa, versicolor, virginica)
- confidence: 예측 신뢰도 (0-1)
- probabilities: 각 클래스별 확률
- feature_importance: 특성별 중요도

모델 정보:
- 타입: {self.metadata['model_type']}
- 정확도: {self.metadata['accuracy']:.4f}
- 학습 샘플: {self.metadata['training_samples']}개
"""
        return description.strip()
    
    def validate_input(self, input_data: Dict[str, float]) -> Tuple[bool, str]:
        """입력 데이터의 유효성을 검증한다"""
        required_fields = self.skill['input_schema']['required']
        properties = self.skill['input_schema']['properties']
        
        for field in required_fields:
            if field not in input_data:
                return False, f"필수 필드 누락: {field}"
        
        for field, value in input_data.items():
            if field in properties:
                prop = properties[field]
                if 'minimum' in prop and value < prop['minimum']:
                    return False, f"{field}가 최소값({prop['minimum']})보다 작다: {value}"
                if 'maximum' in prop and value > prop['maximum']:
                    return False, f"{field}가 최대값({prop['maximum']})보다 크다: {value}"
        
        return True, "유효한 입력"
    
    def predict(self, input_data: Dict[str, float]) -> Dict[str, Any]:
        """예측을 수행한다"""
        is_valid, message = self.validate_input(input_data)
        if not is_valid:
            raise ValueError(message)
        
        feature_names = self.metadata['feature_names']
        X = np.array([[
            input_data['sepal_length'],
            input_data['sepal_width'],
            input_data['petal_length'],
            input_data['petal_width']
        ]])
        
        prediction = self.model.predict(X)[0]
        probabilities = self.model.predict_proba(X)[0]
        
        target_names = self.metadata['target_names']
        predicted_class = target_names[prediction]
        confidence = float(probabilities[prediction])
        
        feature_importance = dict(zip(
            feature_names,
            self.model.feature_importances_.tolist()
        ))
        
        return {
            "predicted_class": predicted_class,
            "confidence": confidence,
            "probabilities": dict(zip(target_names, probabilities.tolist())),
            "feature_importance": feature_importance,
            "input_features": input_data,
            "timestamp": datetime.now().isoformat()
        }
    
    def explain_prediction(self, result: Dict[str, Any]) -> str:
        """예측 결과를 자연어로 설명한다"""
        messages = [
            {
                "role": "system",
                "content": "당신은 머신러닝 예측 결과를 비전문가도 이해할 수 있도록 설명하는 전문가다. 예측 결과를 자연스러운 한국어로 설명한다. 문장은 ~다로 끝낸다."
            },
            {
                "role": "user",
                "content": f"""
다음 붓꽃 분류 예측 결과를 설명해달라:

입력 특성:
- 꽃받침 길이: {result['input_features']['sepal_length']} cm
- 꽃받침 너비: {result['input_features']['sepal_width']} cm
- 꽃잎 길이: {result['input_features']['petal_length']} cm
- 꽃잎 너비: {result['input_features']['petal_width']} cm

예측 결과:
- 예측된 품종: {result['predicted_class']}
- 신뢰도: {result['confidence']:.2%}

각 품종별 확률:
{json.dumps(result['probabilities'], indent=2, ensure_ascii=False)}

특성별 중요도:
{json.dumps(result['feature_importance'], indent=2, ensure_ascii=False)}

위 결과를 바탕으로 다음을 포함하여 설명해달라:
1. 예측된 품종과 그 신뢰도
2. 왜 이 품종으로 예측되었는지 (중요한 특성 위주로)
3. 다른 품종일 가능성은 얼마나 되는지
"""
            }
        ]
        
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.7,
            max_tokens=500
        )
        
        return response.choices[0].message.content
    
    def generate_tool_definition(self) -> Dict[str, Any]:
        """OpenAI Function Calling을 위한 도구 정의를 생성한다"""
        return {
            "type": "function",
            "function": {
                "name": "predict_iris_species",
                "description": self.skill['description'],
                "parameters": {
                    "type": "object",
                    "properties": {
                        "sepal_length": {
                            "type": "number",
                            "description": "꽃받침 길이 (cm)"
                        },
                        "sepal_width": {
                            "type": "number",
                            "description": "꽃받침 너비 (cm)"
                        },
                        "petal_length": {
                            "type": "number",
                            "description": "꽃잎 길이 (cm)"
                        },
                        "petal_width": {
                            "type": "number",
                            "description": "꽃잎 너비 (cm)"
                        }
                    },
                    "required": ["sepal_length", "sepal_width", "petal_length", "petal_width"]
                }
            }
        }

print("IrisEnableAgent 클래스 정의 완료")

IrisEnableAgent 클래스 정의 완료


---
## 4. Enable Agent 초기화 및 테스트

In [9]:
agent = IrisEnableAgent('skills/iris_agent_skill.yaml')

print("\n" + "="*60)
print("Enable Agent 기능 설명")
print("="*60)
print(agent.get_capability_description())

Enable Agent 초기화 완료: Iris Classification Agent
모델 정확도: 0.9333

Enable Agent 기능 설명
Agent 이름: Iris Classification Agent
버전: 1.0.0
설명: 붓꽃(Iris) 품종을 분류하는 머신러닝 Enable Agent

주요 기능: iris_classification
부가 기능: feature_analysis, prediction_explanation

입력 파라미터:
- sepal_length: 꽃받침 길이 (cm, 4.0-8.0)
- sepal_width: 꽃받침 너비 (cm, 2.0-5.0)
- petal_length: 꽃잎 길이 (cm, 1.0-7.0)
- petal_width: 꽃잎 너비 (cm, 0.1-3.0)

출력:
- predicted_class: 예측된 붓꽃 품종 (setosa, versicolor, virginica)
- confidence: 예측 신뢰도 (0-1)
- probabilities: 각 클래스별 확률
- feature_importance: 특성별 중요도

모델 정보:
- 타입: RandomForestClassifier
- 정확도: 0.9333
- 학습 샘플: 120개


---
## 5. 예측 수행 테스트

3가지 품종에 대한 예측을 수행하고 결과를 비교한다.

In [10]:
test_cases = [
    {
        "name": "Setosa 예제",
        "input": {"sepal_length": 5.1, "sepal_width": 3.5, "petal_length": 1.4, "petal_width": 0.2}
    },
    {
        "name": "Versicolor 예제",
        "input": {"sepal_length": 6.4, "sepal_width": 3.2, "petal_length": 4.5, "petal_width": 1.5}
    },
    {
        "name": "Virginica 예제",
        "input": {"sepal_length": 6.3, "sepal_width": 3.3, "petal_length": 6.0, "petal_width": 2.5}
    }
]

results = []

for test_case in test_cases:
    print("\n" + "="*60)
    print(f"{test_case['name']}")
    print("="*60)
    
    result = agent.predict(test_case['input'])
    results.append(result)
    
    print(f"\n예측 결과:")
    print(f"  품종: {result['predicted_class']}")
    print(f"  신뢰도: {result['confidence']:.2%}")
    
    print(f"\n각 품종별 확률:")
    for species, prob in result['probabilities'].items():
        bar = '#' * int(prob * 50)
        print(f"  {species:12s}: {bar} {prob:.2%}")
    
    print(f"\n특성 중요도:")
    sorted_importance = sorted(result['feature_importance'].items(), key=lambda x: x[1], reverse=True)
    for feature, importance in sorted_importance:
        bar = '#' * int(importance * 50)
        print(f"  {feature:30s}: {bar} {importance:.4f}")


Setosa 예제

예측 결과:
  품종: setosa
  신뢰도: 100.00%

각 품종별 확률:
  setosa      : ################################################## 100.00%
  versicolor  :  0.00%
  virginica   :  0.00%

특성 중요도:
  petal width (cm)              : ##################### 0.4381
  petal length (cm)             : ##################### 0.4316
  sepal length (cm)             : ##### 0.1160
  sepal width (cm)              :  0.0142

Versicolor 예제

예측 결과:
  품종: versicolor
  신뢰도: 97.90%

각 품종별 확률:
  setosa      :  0.00%
  versicolor  : ################################################ 97.90%
  virginica   : # 2.10%

특성 중요도:
  petal width (cm)              : ##################### 0.4381
  petal length (cm)             : ##################### 0.4316
  sepal length (cm)             : ##### 0.1160
  sepal width (cm)              :  0.0142

Virginica 예제

예측 결과:
  품종: virginica
  신뢰도: 100.00%

각 품종별 확률:
  setosa      :  0.00%
  versicolor  :  0.00%
  virginica   : ################################################## 100.00%

특성 

---
## 6. AI 설명 생성

OpenAI API를 사용하여 예측 결과를 자연어로 설명한다.

In [11]:
print("\n" + "="*60)
print("AI 설명 생성")
print("="*60)

for i, (test_case, result) in enumerate(zip(test_cases, results), 1):
    print(f"\n[{i}] {test_case['name']}")
    print("-" * 60)
    
    explanation = agent.explain_prediction(result)
    print(explanation)
    print()


AI 설명 생성

[1] Setosa 예제
------------------------------------------------------------
예측 결과에 따르면, 입력된 붓꽃의 특성으로부터 'setosa'라는 품종이 예측되었다. 이 예측의 신뢰도는 100.00%로 매우 높은 수치다. 이는 모델이 입력된 특성을 바탕으로 확신을 가지고 'setosa' 품종으로 분류했음을 의미한다.

예측이 'setosa'로 나온 이유는 주로 꽃잎의 길이와 너비가 큰 영향을 미쳤기 때문이다. 특성별 중요도를 살펴보면, 꽃잎 길이(1.4 cm)와 꽃잎 너비(0.2 cm)가 가장 중요한 특성으로 나타났으며, 각각 43.16%와 43.81%의 중요도를 가지고 있다. 반면에 꽃받침 길이(5.1 cm)와 너비(3.5 cm)는 상대적으로 낮은 중요도를 보였기 때문에, 이 두 특성은 예측에 큰 영향을 미치지 않았다.

또한, 다른 품종인 'versicolor'와 'virginica'에 대한 예측 확률은 각각 0.0%로 나타났다. 이는 이 두 품종의 가능성이 전혀 없음을 의미하며, 입력된 특성으로는 오직 'setosa'만이 적합하다는 결론을 내릴 수 있다. 따라서, 주어진 특성을 기반으로 할 때, 이 붓꽃은 확실히 'setosa' 품종이라고 볼 수 있다.


[2] Versicolor 예제
------------------------------------------------------------
입력된 붓꽃의 특성을 바탕으로 예측된 품종은 'versicolor'이며, 이 예측의 신뢰도는 97.90%이다. 이는 매우 높은 확신을 가지고 이 품종으로 분류되었음을 의미한다.

예측이 'versicolor'로 나온 이유는 꽃잎의 길이와 너비가 이 품종의 특성에서 중요한 역할을 하기 때문이다. 특성별 중요도를 살펴보면, 꽃잎 길이(4.5 cm)와 꽃잎 너비(1.5 cm)가 각각 43.16%와 43.81%의 비율로 가장 높은 중요도를 차지하고 있다. 이는 꽃잎의 특성이 붓꽃 품종을 구별하는 

---
## 7. OpenAI Function Calling 통합

Enable Agent를 OpenAI Function Calling 도구로 등록하여 자연어 인터페이스를 제공한다.

In [12]:
tool_definition = agent.generate_tool_definition()

print("OpenAI Function Calling Tool 정의:")
print(json.dumps(tool_definition, indent=2, ensure_ascii=False))

OpenAI Function Calling Tool 정의:
{
  "type": "function",
  "function": {
    "name": "predict_iris_species",
    "description": "붓꽃(Iris) 품종을 분류하는 머신러닝 Enable Agent",
    "parameters": {
      "type": "object",
      "properties": {
        "sepal_length": {
          "type": "number",
          "description": "꽃받침 길이 (cm)"
        },
        "sepal_width": {
          "type": "number",
          "description": "꽃받침 너비 (cm)"
        },
        "petal_length": {
          "type": "number",
          "description": "꽃잎 길이 (cm)"
        },
        "petal_width": {
          "type": "number",
          "description": "꽃잎 너비 (cm)"
        }
      },
      "required": [
        "sepal_length",
        "sepal_width",
        "petal_length",
        "petal_width"
      ]
    }
  }
}


In [13]:
def demo_function_calling(user_query: str):
    """자연어 쿼리로 Agent 호출 데모"""
    print(f"\n사용자: {user_query}")
    print("-" * 60)
    
    client = OpenAI()
    
    messages = [
        {"role": "system", "content": "당신은 붓꽃 품종 분류를 도와주는 AI 어시스턴트다."},
        {"role": "user", "content": user_query}
    ]
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=[tool_definition],
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    
    if response_message.tool_calls:
        tool_call = response_message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        
        print(f"Agent 호출: predict_iris_species")
        print(f"입력: {json.dumps(function_args, indent=2, ensure_ascii=False)}")
        
        result = agent.predict(function_args)
        
        messages.append(response_message)
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": tool_call.function.name,
            "content": json.dumps(result, ensure_ascii=False)
        })
        
        final_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
        
        print(f"\nAI 응답:")
        print(final_response.choices[0].message.content)
    else:
        print(f"AI 응답:")
        print(response_message.content)

demo_function_calling(
    "꽃받침 길이 5.8cm, 꽃받침 너비 2.7cm, 꽃잎 길이 5.1cm, 꽃잎 너비 1.9cm인 붓꽃은 어떤 품종일까?"
)


사용자: 꽃받침 길이 5.8cm, 꽃받침 너비 2.7cm, 꽃잎 길이 5.1cm, 꽃잎 너비 1.9cm인 붓꽃은 어떤 품종일까?
------------------------------------------------------------
Agent 호출: predict_iris_species
입력: {
  "sepal_length": 5.8,
  "sepal_width": 2.7,
  "petal_length": 5.1,
  "petal_width": 1.9
}

AI 응답:
해당 붓꽃은 "virginica" 품종으로 분류되었습니다. 이 품종으로 예측된 확신도는 100%입니다. 주요한 특징으로는 꽃잎의 길이와 너비가 중요한 역할을 합니다.
